In [ ]:
from torch.nn.modules.activation import ReLU

import torch
import torch.nn as nn

In [ ]:
def conv_bn_relu(in_channels, out_channels, kernel_size=3, stride=1, padding=0):
    """
    합성곱 연산, Batch Normalization, ReLU 활성함수를 연속적으로 거치도록 하는 Sequential을 return하는 함수

    매개변수(Parameters)
    ----------------------
    in_channels: int형, 입력으로 들어오는 이미지의 채널 개수
    out_channels: int형, 출력으로 반환할 이미지의 채널 개수
    kernel_size: int 혹은 tuple 형, 사용할 필터의 크기 정보, default 값 3
    stride: int 혹은 tuple 형, 스트라이드 값, defalut 값 1
    padding: int 혹은 tuple 혹은 str형, 패딩에 대한 정보, defalut 값 0(패딩 없음)

    반환 값(Returns)
    ----------------------
    합성곱 연산, Batch Normalization, ReLU 활성함수를 연속적으로 거치도록 하는 torch.nn.Sequential
    """
    return nn.Sequential(
        # 입력받은 매개변수에 따라, 합성곱 연산을 진행
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
        # 배치 정규화 진행
        nn.BatchNorm2d(out_channels),
        # 활성함수인 ReLU 함수 거치기
        nn.ReLU(inplace=True)
    )

In [ ]:
def SeparableConv2D(in_channels, out_channels, kernel=3):
    """
    Separable Convolution 연산을 진행하는 Sequential을 return하는 함수

    매개변수(Parameters)
    ----------------------
    in_channels: int형, 입력으로 들어오는 이미지의 채널 개수
    out_channels: int형, 출력으로 반환할 이미지의 채널 개수
    kernel: int 혹은 tuple 형, 사용할 필터의 크기 정보, default 값 3
    
    반환 값(Returns)
    ----------------------
    Separable Convolution 연산을 진행하는 Sequential
    """
    return nn.Sequential(
        # 입력받은 채널의 개수를 보존하는 형태로, 채널 수가 1인 필터를 합성곱 연산
        nn.Conv2d(in_channels, in_channels, kernel_size=kernel, stride=1, groups=in_channels, padding=1, bias=False),
        # 1x1 합성곱 연산 진행, 채널 개수를 원하는 출력 채널 개수로 조정
        nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
    )

In [ ]:
class ResidualXceptionBlock(nn.Module):
    """
    MiniXception 구조의 핵심인 부분으로, 잔차 연결로 두 경로의 계층 연산을 합해주는 클래스
    """
    def __init__(self, in_channels, out_channels, kernel=3):
        """
        필요한 계층을 정의하는 부분들이 담긴 생성자(constructor)

        매개변수(Parameters)
        ----------------------
        in_channels: int형, 입력으로 들어오는 이미지의 채널 개수
        out_channels: int형, 출력으로 반환할 이미지의 채널 개수
        kernel: int 혹은 tuple 형, 사용할 필터의 크기 정보, default 값 3
        """
        # 상속받은 nn.Module 클래스의 생성자 호출
        super().__init__()

        # 그림 21에서, ResidualXceptionBlock을 구성하는 4개의 사각형 중 좌측 가장 하단
        # 이 계층에서의 합성곱 연산을 통해 출력 데이터의 채널 수로 조정
        # Separable convolution 연산을 진행하는 계층, Sequential 형태. 앞서 정의한 함수 활용
        self.depthwise_conv1 = SeparableConv2D(in_channels, out_channels, kernel)
        # 배치 정규화 진행
        self.bn1 = nn.BatchNorm2d(out_channels)
        # 활성함수로 ReLU 함수를 사용
        self.relu1 = nn.ReLU(inplace=True)

        # 그림 21에서, ResidualXceptionBlock을 구성하는 4개의 사각형 중 좌측 가운데
        # 앞선 계층에서 출력 데이터의 채널 개수를 맞추었으므로, 채널 개수에는 변화를 주지 않음
        # Separable convolution 연산을 진행하는 계층, Sequential 형태. 앞서 정의한 함수 활용
        self.depthwise_conv2 = SeparableConv2D(out_channels, out_channels, kernel)
        # 배치 정규화 진행
        self.bn2 = nn.BatchNorm2d(out_channels)

        # 그림 21에서, ResidualXceptionBlock을 구성하는 4개의 사각형 중 좌측 상단
        # 최대 풀링 진행
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        
        # 그림 21에서, ResidualXceptionBlock을 구성하는 4개의 사각형 중 우측
        # 잔차 연결을 진행하기 위한 갈래의 계층을 구현
        # 합성곱 연산 진행, 필요한 출력 데이터의 채널 수가 되도록 조정
        self.residual_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        # 배치 정규화 진행
        self.residual_bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        """
        순전파를 진행하도록 하는 함수

        매개변수(Parameters)
        ----------------------
        x: Tensor, 입력 데이터
        """
        # 그림 21에서, ResidualXceptionBlock을 구성하는 두 갈래 중 우측
        # 우측 갈래를 따른 순전파 진행 결과를 residual에 저장
        residual = self.residual_conv(x)
        residual = self.residual_bn(residual)
        
        # 그림 21에서, ResidualXceptionBlock을 구성하는 두 갈래 중 좌측
        # 우측 갈래를 따른 순전파 진행 결과를 x에 저장
        x = self.depthwise_conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.depthwise_conv2(x)
        x = self.bn2(x)
        x = self.maxpool(x)

        # 두 갈래의 순전파 결과를 합한 Tensor 결과를 반환하여 잔차 연결 구현
        return x + residual

In [ ]:
class Mini_Xception(nn.Module):
    """
    전체 MiniXception 구조를 구현한 클래스
    """
    def __init__(self):
        """
        필요한 계층을 정의하는 부분들이 담긴 생성자(constructor)
        """
        # 상속받은 nn.Module 클래스의 생성자 호출
        super().__init__()

        # 그림 21에서, ResidualXceptionBlock 이전의 합성곱-배치 정규화-ReLU 계층
        self.conv1 = conv_bn_relu(1, 8, kernel_size=3, stride=1, padding=0)
        self.conv2 = conv_bn_relu(8, 8, kernel_size=3, stride=1, padding=0)
        
        # 그림 21에서, ResidualXceptionBlock 4개를 모아둔 부분
        # 순전파가 진행될수록, 이미지 채널 개수를 2배씩 증가
        # 이전 계층의 출력 채널 개수가 다음 채널의 입력 채널의 개수와 동일하도록 설정 필요
        # 채널 개수를 증가시키는 점에 대한 기본적인 아이디어가 궁금하다면, VGGNet이라는 구조에 대해 찾아보자.
        self.residual_blocks = nn.ModuleList([
            ResidualXceptionBlock(8 , 16),
            ResidualXceptionBlock(16, 32),
            ResidualXceptionBlock(32, 64),
            ResidualXceptionBlock(64, 128)            
        ])

        # 그림 21에서, ResidualXceptionBlock 4개를 거친 이후의 부분
        # 합성곱 계층
        # Global Average Pooling 단계로 넘어가기 직전의 채널 수를 7로 설정함에 주목할 것.
        self.conv3 = nn.Conv2d(128, 7, kernel_size=3, stride=1, padding=1)
        # Global Average Pooling 계층
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        """
        순전파를 진행하도록 하는 함수

        매개변수(Parameters)
        ----------------------
        x: Tensor, 입력 데이터
        """        
        # 그림 21에서, ResidualXceptionBlock 이전의 순전파
        x = self.conv1(x)
        x = self.conv2(x)

        # 그림 21에서, ResidualXceptionBlock 4개에 대한 순전파
        # ModuleList를 순회하면서, 저장된 계층들을 차례로 적용하도록 함
        for block in self.residual_blocks:
            x = block(x)

        # 그림 21에서, ResidualXceptionBlock 이후의 합성곱 계층
        x = self.conv3(x)

        # 그림 21에서, Global Average Pooling
        x = self.global_avg_pool(x)

        return x

In [ ]:
if __name__ == '__main__':
    x = torch.randn((2, 1, 48, 48))
    model = Mini_Xception()
    y = model(x)
    print(y.squeeze().shape)